# Clustering

Vamos a realizar operaciones de **clustering** (segmentación) de datos.
La idea es encontrar una estructura dentro de un dataset donde originalmente no la había.
No se tiene un objetivo de predicción (se trata **aprendizaje no supervisado**), sino de uno de entendimiento de los datos a través del particionamiento del dataset en grupos de instancias.

# Parte 1. K-Means con datos sintéticos

Para poder entender como se utilizan los algoritmos de clustering, vamos inicialmente a crear un dataset sintético con datos ficticios que nos permita ilustrar los aspectos de llamado a los métodos de python.

Vamos a generar y visualizar en un plot 300 puntos aleatorios distribuidos alrededor de 4 centros en un espacio bidimensional, con una desviación estándar de 0.7. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import math
from collections import Counter


from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score, silhouette_samples, silhouette_score, calinski_harabaz_score
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [ ]:
from sklearn.datasets.samples_generator import make_blobs
X, grupo = make_blobs(n_samples=300, centers=4, cluster_std=0.8, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);

En X van a quedar las coordenadas de los puntos y en **grupo** los clusters originales a los que pertencen los datos.

In [ ]:
X[0:5]

In [ ]:
grupo[0:5]

El algoritmo de K-Means recibe como parámetro el número de clusters que se buscan (hay que sepecificarlo ya que no lo determina automáticamente). Como sabemos que los datos sintéticos se crearon con 4 grupos, vamos a analizar si K-Means los logra detectar.

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0)
kmeans.fit(X)
grupo_kmeans = kmeans.predict(X)

Vamos a plotear los clusters encontrados con diferentes colores.

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=grupo_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

Ahora comparemos los grupos encontrados con los reales, utilizando una matriz de confusión.

In [ ]:
cm = confusion_matrix(grupo, grupo_kmeans)
plt.imshow(cm, cmap=plt.cm.Blues)
plt.title("Comparación entre los clusters reales y los descubiertos por K-Means")
plt.colorbar()
tick_marks = np.arange(4)
plt.xticks(tick_marks, ['0','1','2','3'])
plt.yticks(tick_marks, ['0','1','2','3'])
cm

En las filas encontramos los grupos reales y en las columnas los de K-Means. Encontramos 6 errores, todos asociados por k-means al grupo 0 cuando eran de alguno de los otros 3 grupos.

Hay que tener en cuenta que el orden de los nombres de los grupos generados puede no conincidir con el orden de los grupos encontrados por K-Means, como es el caso aquí.

Lo que vemos es que parece haber una concordancia entre los clusters encontrados por K-Means y los reales: los grupos 0, 1, 2 y 3 de k-means corresponden a los grupos 1, 0, 2, y 3 encontrados por K-Means, respectivamente.

Vamos a cambiar el orden de los clusters de k-means para poder entender mejor los resultados

In [ ]:
traducir = [1, 0, 2, 3]

grupo_kmeans_reorg = []
for g_k, g in zip(grupo_kmeans, grupo):
    grupo_kmeans_reorg.append(traducir[g_k])
print(grupo_kmeans_reorg)

Podemos hacer esto de una manera mas breve utilizando una de las particularidades de Python: List comprehensions, que permite resumir operaciones simples realizadas dentro de un ciclo:

In [ ]:
grupo_kmeans_reorg = [traducir[g_k] for g_k in grupo_kmeans]
print(grupo_kmeans_reorg)

In [ ]:
cm = confusion_matrix(grupo, grupo_kmeans_reorg)
cm

In [ ]:
accuracy_score(grupo, grupo_kmeans_reorg)

Veamos gráficamente cuáles son los registros que se asocian a un grupo diferente a su original.

In [ ]:
diferentes = []
for (x0, x1), g, gk in zip(X, grupo, grupo_kmeans_reorg):
    if g!=gk:
        diferentes.append([x0, x1])
        

In [ ]:
X[0:5]

In [ ]:
diferentes = np.array(diferentes)

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(X[:, 0], X[:, 1], c=grupo_kmeans_reorg, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

plt.scatter(diferentes[:, 0], diferentes[:, 1], c='red', marker="x", s=150)

**Preguntas**:
Comparamos los grupos creados por el clustering con los originales (esto se llama "clasificación no supervisada")
1. Expliquen la primera matriz de confusión obtenida y el por qué fue necesario recodificar los segmentos obtenidos por el clustering.
2. ¿Qué tan bien puede K-Means encontrar las categorías originales en terminos de accuracy?
3. ¿Tiene sentido crear un test set para un clustering?

# Parte 2. K-Means con datos reales

In [ ]:
data = pd.read_csv('01 - ComprasClientes.csv', na_values=".")
print(data.shape)
data.head(5)

In [ ]:
data.describe(include="all")

In [ ]:
data.info()

## Preparación de los datos

**¿Qué problemas saltan a la vista al inspeccionar los datos?**

1. Las variables Channel y Region tienen tipo int64, cuando en realidad codifican categorías de canales y de regiones. Es necesario cambiar sus tipos.
1. Tenemos en todas las variables de consumo valores anormalmente grandes que pueden considerarse excepciones en el mejor de los casos (anomalías o errores de captura en el peor de los casos). Hay que identificar los registros en cuestión y evaluar la posibilidad de descartarlos pues pueden influenciar negativamente muchos de los modelos que se pueden aprender a partir de los datos.
1. Las escalas de las variables que denotan los montos consumidos de cada tipo de productos son muy disparejas. Es necesario normalizar los datos ya que de no hacerlo se otorgaría una importancia demasiado desmedida a variables como Fresh casi que ignorando variables como Delicatessen.

Arreglamos primero los tipos de datos incorrectos:

In [ ]:
data.Channel = data.Channel.astype(str)
data.Region = data.Region.astype(str)
data.info()

Antes de normalizar es necesario limpiar las excepciones o anomalías con valores o muy grandes o muy pequeñas. Vamos a analizar las variables numéricas a partir de diagramas de cajas y bigotes.

In [ ]:
plt.figure(figsize=(12,12))
data.boxplot()

Vemos que hay valores muy importantes en todas las variables. Si contamos los puntos individuales más elevados podemos identificar 6 o menos puntos que sobrepasan la mayoría de los demás.
Puede que algunos de los puntos excepcionales en las diferentes variables correspondan a los mismos individuos. Vamos a identificar los top 6 de valores mas importantes en cada tipo de producto y no los vamos a considerar en los análisis siguientes.

In [ ]:
temp = data.sort_values(['Fresh'], ascending=False)
print("Excepciones de Fresh: ", np.sort(temp[0:6].index.get_values()))
indicesAQuitar = temp[0:6].index.get_values()

temp = data.sort_values(['Milk'], ascending=False)
print("Excepciones de Milk: ", np.sort(temp[0:6].index.get_values()))
indicesAQuitar = np.union1d(indicesAQuitar, temp[0:6].index.get_values())


temp = data.sort_values(['Grocery'], ascending=False)
print("Excepciones de Grocery: ", np.sort(temp[0:6].index.get_values()))
indicesAQuitar = np.union1d(indicesAQuitar, temp[0:6].index.get_values())

temp = data.sort_values(['Frozen'], ascending=False)
print("Excepciones de Frozen: ", np.sort(temp[0:6].index.get_values()))
indicesAQuitar = np.union1d(indicesAQuitar, temp[0:6].index.get_values())

temp = data.sort_values(['Detergents_Paper'], ascending=False)
print("Excepciones de Detergents_Paper: ", np.sort(temp[0:6].index.get_values()))
indicesAQuitar = np.union1d(indicesAQuitar, temp[0:6].index.get_values())

temp = data.sort_values(['Delicassen'], ascending=False)
print("Excepciones de Delicassen: ", np.sort(temp[0:6].index.get_values()))
indicesAQuitar = np.union1d(indicesAQuitar, temp[0:6].index.get_values())

indicesAQuitar

Tenemos 22 registros identificados como excepciones. Vemos que algunos tienen valores excepcionales según diferentes tipos de consumo (23, 47, 61, 65, 85, ...)

In [ ]:
data.shape

In [ ]:
dataDepurado = data.loc[~data.index.isin(indicesAQuitar)]
dataDepurado.shape

Vamos ahora a normalizar los datos para que todas las variables tengan la misma importancia. Solo vamos a considerar los datos numéricos, por lo que no incluimos las variables Channel y Region.

In [ ]:
dataStd = pd.DataFrame(preprocessing.scale(dataDepurado.iloc[:,2:]))
dataStd.columns=dataDepurado.columns[2:]

In [ ]:
dataStd.mean(axis=0)

In [ ]:
dataStd.std(axis=0)

## Clustering

**Con un k de 3, realice un clustering por K-Means (utilicen random_state=0).**

**Agregue una columna "Cluster" con el segmento correspondiente (0, 1, o 2) al dataset.**

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0, n_init=10)
kmeans.fit(dataStd.iloc[:,0:6])

El método KMeans en scikit-learn permite definir los valores de ciertos parámetros que controlan la ejecución del algoritmo de clustering. Nos interesan particularmente:
- **n_clusters**: número de clusters que se desean (el parámetro "K"). Por defecto es 8.
- **init**: el método de inicialización de los centroides. Por defecto es "k-means++". Otros valores son "random" o un array con los centroides iniciales
- **n_init**: número de inicializaciones diferentes a ensayar para evitar llegar a un óptimo local. Por defecto es 10
- **max_iter**: Máximo número de iteraciones que se esparará para llegar a convergencia. Por defecto es 300.
- **tol**: tolerancia para determinar que se ha llegado o no a convergenia con respecto a la reducción del WSS (interia). Por defecto es 0.0001
- **random_state**: semilla de inicialización del generador pseudo-aleatorio para poder reproducir los resultados.

El objeto resultado del KMeans después de lanzado el ajuste del algoritmo consta de diferentes valores de salida:
- **cluster_centers_**: los centroides finales de los clusters.
- **labels_**: los clusters a los cuales termina perteneciendo cada instancia del set de aprendizaje.
- **interia_**: el WSS final.
- **n_iter_**: el número de iteraciones que tomó llegar a convergencia.

In [ ]:
print("Le tomó a KMeans", kmeans.n_iter_, "iteraciones llegar a convergencia, con un WSS final de:n",
      kmeans.inertia_, "y los centroides siguientes:", kmeans.cluster_centers_)

In [ ]:
kmeans.labels_

También podemos utilizar el objeto kmeans resultante como modelo de clasificación, al que a través del método *predict* se le puede enviar un dataset para evaluar y obtener los clusters a los que pertenecen. Por ejemplo, utilicemoslo para clasificar los mismos ejemplos de entrenamiento:

In [ ]:
clusters = kmeans.predict(dataStd.iloc[:, 0:6])
clusters

In [ ]:
counter=Counter(clusters)
print(counter)

Tenemos 3 clusters de 238, 93 y 87 instancias cada uno.
Agregamos una columna al dataframe con los datos analizados que indique a que cluster pertenece cada registro.

In [ ]:
dataStd.loc[:,'Cluster'] = clusters

In [ ]:
dataStd.columns

In [ ]:
dataStd[0:5]

# Parte 3. Interpretación de los clusters

## Interpretación de los clusters, con k =3

Vamos a tratar de entender cuales son las características de los registros que los componen. Para ello vamos a ver gráficos de densidad que permitan identificar las predilecciones de compras de los clientes que pertenecen a cada cluster.

In [ ]:
var_num = ['Fresh', 'Milk', 'Grocery', 'Frozen', 'Detergents_Paper', 'Delicassen']

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
fig = plt.figure(figsize=(18,15))
i=1
for var in var_num:
    ax = fig.add_subplot(math.ceil(len(var_num)/2), 2, i)
    sns.kdeplot(dataStd.loc[dataStd.Cluster==0][var], shade=True, color='r', ax=ax);
    sns.kdeplot(dataStd.loc[dataStd.Cluster==1][var], shade=True, color='g', ax=ax);
    sns.kdeplot(dataStd.loc[dataStd.Cluster==2][var], shade=True, color='b', ax=ax);
    plt.title(var)
    plt.legend(['Cluster 0', 'Cluster 1', 'Cluster 2'])
    i+=1

Veámoslos en scatterplots para entender mejor las diferencias:

In [ ]:
fig = plt.figure(figsize=(15,15))
colorPalette = ["r", "g", "b"]
ax = fig.add_subplot(2, 2, 1)
sns.scatterplot(x="Fresh", y="Milk", hue="Cluster", data=dataStd, ax=ax, palette=colorPalette, s=100, alpha=0.5)
plt.title("Fresh vs. Milk")
ax = fig.add_subplot(2, 2, 2)
sns.scatterplot(x="Frozen", y="Grocery", hue="Cluster", data=dataStd, ax=ax, palette=colorPalette, s=100, alpha=0.5)
plt.title("Frozen vs. Grocery")
ax = fig.add_subplot(2, 2, 3)
sns.scatterplot(x="Delicassen", y="Detergents_Paper", hue="Cluster", data=dataStd, ax=ax, palette=colorPalette, s=100, alpha=0.5)
plt.title("Delicassen vs. Detergents_Paper")
ax = fig.add_subplot(2, 2, 4)
sns.scatterplot(x="Fresh", y="Frozen", hue="Cluster", data=dataStd, ax=ax, palette=colorPalette, s=100, alpha=0.5)
plt.title("Fresh vs. Frozen")
plt.show()

Cómo se puede distinguir entre:
- los rojos y los demás
- los verdes y los demás
- los azules y los demás

Podemos interpretar entonces los clusters de esta manera:
- Cluster 0 (Rojo, 238 registros). Tiene valores:
    - Altos : Milk, Grocery, Detergents_Paper
    - Medios: Delicassen
    - Bajos : Fresh, Frozen
- Cluster 1 (Verde, 93 registros). Tiene valores:
    - Altos : 
    - Medios: 
    - Bajos : Fresh, Milk, Grocery, Frozen, Detergents_Paper, Delicassen
- Cluster 2 (Azul, 87 registros). Tiene valores:
    - Altos : Fresh, Frozen
    - Medios: Delicassen
    - Bajos : Milk , Grocery, Detergents_Paper
    

**Qué podemos decir de los 3 clusters, qué adjetivo les darían para describirlos?**

## Interpretación de los clusters, con k = 4

Repetimos el análisis con k=4

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0, n_init=10)
kmeans.fit(dataStd.iloc[:, 0:6])
clusters = kmeans.labels_

In [ ]:
dataStd['Cluster']= clusters
counter=Counter(clusters)
print(counter)

In [ ]:
fig = plt.figure(figsize=(18,15))
i=1
for var in var_num:
    ax = fig.add_subplot(math.ceil(len(var_num)/2), 2, i)
    sns.kdeplot(dataStd.loc[dataStd.Cluster==0][var], shade=True, color='r', ax=ax);
    sns.kdeplot(dataStd.loc[dataStd.Cluster==1][var], shade=True, color='g', ax=ax);
    sns.kdeplot(dataStd.loc[dataStd.Cluster==2][var], shade=True, color='b', ax=ax);
    sns.kdeplot(dataStd.loc[dataStd.Cluster==3][var], shade=True, color='y', ax=ax);
    plt.title(var)
    plt.legend(['Cluster 0', 'Cluster 1', 'Cluster 2', 'Cluster 3'])
    i+=1

Veamoslos en scatterplots para entender mejor las diferencias:

In [ ]:
fig = plt.figure(figsize=(15,15))
colorPalette = ["r", "g", "b", "y"]
ax = fig.add_subplot(2, 2, 1)
sns.scatterplot(x="Fresh", y="Milk", hue="Cluster", data=dataStd, ax=ax, palette=colorPalette, s=100, alpha=0.5)
plt.title("Fresh vs. Milk")
ax = fig.add_subplot(2, 2, 2)
sns.scatterplot(x="Frozen", y="Grocery", hue="Cluster", data=dataStd, ax=ax, palette=colorPalette, s=100, alpha=0.5)
plt.title("Frozen vs. Grocery")
ax = fig.add_subplot(2, 2, 3)
sns.scatterplot(x="Delicassen", y="Detergents_Paper", hue="Cluster", data=dataStd, ax=ax, palette=colorPalette, s=100, alpha=0.5)
plt.title("Delicassen vs. Detergents_Paper")
ax = fig.add_subplot(2, 2, 4)
sns.scatterplot(x="Fresh", y="Frozen", hue="Cluster", data=dataStd, ax=ax, palette=colorPalette, s=100, alpha=0.5)
plt.title("Fresh vs. Frozen")

Cómo se puede distinguir entre:
- los rojos y los demás
- los verdes y los demás
- los azules y los demás
- los amarillos y los demás

## Interpretación de los clusters, con k =2

Repetimos el análisis con k=2

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0, n_init=10)
kmeans.fit(dataStd.iloc[:, 0:6])
clusters = kmeans.labels_

In [ ]:
dataStd['Cluster']= clusters
counter=Counter(clusters)
print(counter)

In [ ]:
fig = plt.figure(figsize=(18,15))
i=1
for var in var_num:
    ax = fig.add_subplot(math.ceil(len(var_num)/2), 2, i)
    sns.kdeplot(dataStd.loc[dataStd.Cluster==0][var], shade=True, color='r', ax=ax);
    sns.kdeplot(dataStd.loc[dataStd.Cluster==1][var], shade=True, color='b', ax=ax);
    plt.title(var)
    plt.legend(['Cluster 0', 'Cluster 1'])
    i+=1

Vemos que con K=2, las compras en Delicatessen y Frozen no sirven para discriminar entre los 2 grupos.

Veamoslos en scatterplots para entender mejor las diferencias:

In [ ]:
fig = plt.figure(figsize=(15,7))
colorPalette = ["r", "b"]
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(x="Fresh", y="Milk", hue="Cluster", data=dataStd, ax=ax, palette=colorPalette, s=100, alpha=0.5)
plt.title("Fresh vs. Milk")
ax = fig.add_subplot(1, 2, 2)
sns.scatterplot(x="Detergents_Paper", y="Grocery", hue="Cluster", data=dataStd, ax=ax, palette=colorPalette, s=100, alpha=0.5)
plt.title("Detergents_Paper vs. Grocery")
plt.show()

Vemos que con K=2, las compras en Delicatessen y Frozen no sirven para discriminar entre los 2 grupos.

Podemos interpretar entonces los clusters de esta manera:
- Cluster 0 (Rojo, 330 registros). Tiene valores:
    - Altos : Milk, Grocery, Detergents_Paper
    - Medios: 
    - Bajos : Fresh
- Cluster 1 (Azul, 88 registros). Tiene valores:
    - Altos : Fresh
    - Medios: 
    - Bajos : Milk , Grocery, Detergents_Paper    

Son mucho mas separables las categorías cuando K=2, pero puede que la información no sea suficientemente rica para las acciones que se deseen.

# Parte 4. Determinación del K

### Codo


En el atributo *inertia_* queda el valor de la suma de las distancias cuadráticas entre cada punto y el centro del cluster al que pertenece (el **WSS** - Within Sum of Squares, también llamado más genéricamente **SSE** - Sum of Sqaured Errors)

In [ ]:
kmeans.inertia_
kmeans.init

A partir de los valores de WSS se puede crear el plot a partir del cual se aplica la técnica del codo, creando un clustering para diferentes valores de K. Veamos, según el método del codo, cual sería el valor del K en este conjunto de datos:

In [ ]:
WSSs = []
for i in range(1,15) :
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(dataStd)
    WSSs.append(km.inertia_)
WSSs

In [ ]:
plt.plot(range(1, 15), WSSs)

Con K=3 encontramos aproximadamente el codo. 

### Silueta

Veamos ahora con el método de silueta cuántos clusters deberíamos tener. Obtengamos las siluetas para k = 2, 3, 4 y 5.

Veamos las siluetas de los puntos de cada cluster.
Vamos a crear un bar plot horizontal (barh) para los puntos de cada cluster.

In [ ]:
k=2
kmeans = KMeans(n_clusters=k, random_state=0, n_init=10)
kmeans.fit(dataStd)
y_clusters = kmeans.labels_
cluster_labels = np.unique(y_clusters)

silueta_puntos= silhouette_samples(dataStd, y_clusters, metric='euclidean')

y_ax_lower, y_ax_upper = 0, 0
yticks = []
colores = ['r', 'g', 'b', 'y', 'o']
for i, c in enumerate(cluster_labels):
    silueta_puntos_c = silueta_puntos[y_clusters == c]
    silueta_puntos_c.sort()
    y_ax_upper += len(silueta_puntos_c)
    color = colores[i]
    plt.barh(range(y_ax_lower, y_ax_upper), silueta_puntos_c, height=1.0, 
             edgecolor='none', color=color)

    yticks.append((y_ax_lower + y_ax_upper) / 2.)
    y_ax_lower += len(silueta_puntos_c)
    
silueta_promedio = np.mean(silueta_puntos)
plt.axvline(silueta_promedio, color="black", linestyle="--") 

plt.yticks(yticks, cluster_labels + 1)
plt.ylabel('Cluster')
plt.xlabel('Coeficiente de silueta')

plt.tight_layout()
# plt.savefig('./figures/silhouette.png', dpi=300)
plt.show()

In [ ]:
k=4
kmeans = KMeans(n_clusters=k, random_state=0, n_init=10)
kmeans.fit(dataStd)
y_clusters = kmeans.labels_
cluster_labels = np.unique(y_clusters)

silueta_puntos= silhouette_samples(dataStd, y_clusters, metric='euclidean')

y_ax_lower, y_ax_upper = 0, 0
yticks = []
colores = ['r', 'g', 'b', 'y', 'o']
for i, c in enumerate(cluster_labels):
    silueta_puntos_c = silueta_puntos[y_clusters == c]
    silueta_puntos_c.sort()
    y_ax_upper += len(silueta_puntos_c)
    color = colores[i]
    plt.barh(range(y_ax_lower, y_ax_upper), silueta_puntos_c, height=1.0, 
             edgecolor='none', color=color)

    yticks.append((y_ax_lower + y_ax_upper) / 2.)
    y_ax_lower += len(silueta_puntos_c)
    
silueta_promedio = np.mean(silueta_puntos)
plt.axvline(silueta_promedio, color="black", linestyle="--") 

plt.yticks(yticks, cluster_labels + 1)
plt.ylabel('Cluster')
plt.xlabel('Coeficiente de silueta')

plt.tight_layout()
# plt.savefig('./figures/silhouette.png', dpi=300)
plt.show()

In [ ]:
k=5
kmeans = KMeans(n_clusters=k, random_state=0, n_init=10)
kmeans.fit(dataStd)
y_clusters = kmeans.labels_
cluster_labels = np.unique(y_clusters)

silueta_puntos= silhouette_samples(dataStd, y_clusters, metric='euclidean')

y_ax_lower, y_ax_upper = 0, 0
yticks = []
colores = ['red', 'g', 'b', 'y', 'darkorange']
for i, c in enumerate(cluster_labels):
    silueta_puntos_c = silueta_puntos[y_clusters == c]
    silueta_puntos_c.sort()
    y_ax_upper += len(silueta_puntos_c)
    color = colores[i]
    plt.barh(range(y_ax_lower, y_ax_upper), silueta_puntos_c, height=1.0, 
             edgecolor='none', color=color)

    yticks.append((y_ax_lower + y_ax_upper) / 2.)
    y_ax_lower += len(silueta_puntos_c)
    
silueta_promedio = np.mean(silueta_puntos)
plt.axvline(silueta_promedio, color="black", linestyle="--") 

plt.yticks(yticks, cluster_labels + 1)
plt.ylabel('Cluster')
plt.xlabel('Coeficiente de silueta')

plt.tight_layout()
# plt.savefig('./figures/silhouette.png', dpi=300)
plt.show()

Con el método de silueta lo más indicado podría ser solo considerar dos clusters.

## Calinski-Harabaz

Intentemos ahora con la métrica de Calinski-Harabasz

In [ ]:
CHs = []
for i in range(2,15) :
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(dataStd)
    CH = calinski_harabaz_score(dataStd, km.labels_) 
    CHs.append(CH)
CHs

In [ ]:
plt.plot(range(2, 15), CHs)

Para este criterio, el mayor score está en K=2, al igual que con el método de silueta. Pero como ya dijimos, no nos convendría mucho un clustering con solo dos segmentos para este caso de aplicación.
Encontramos que entre mas pequeño el K mejor, pero vemos que después de K=6 encontramos una gran desmejoría.

# Parte 5. Reducción de dimensionalidad con PCA

Vamos a buscar una mejor representación de los datos que nos permita conservar la mayor cantidad de información a través de la transformación de las 6 variables originales en componentes principales.

In [ ]:
pca = PCA()
pca.fit(dataStd.iloc[:, 0:6])

Una vez ajustado el objeto PCA a un dataset, este permite acceder a diferentes aspectos resultantes de la transformación:
- components_: los ejes de los componentes principales en función de las variables originales. Como teníamos 6 variables, vamos a tener 6 PCs, cada uno con las cargas (*loadings*) correspondientes a cada variable original.

In [ ]:
pca.components_

- explained_variance_: la varianza explicada por cada eje en las unidades originales

In [ ]:
pca.explained_variance_

- explained_variance_ratio_: la proporción de la varianza explicada por cada eje, en porcentaje (la suma da 100%).

In [ ]:
var_exp=pca.explained_variance_ratio_ # varianza explicada por cada PC
cum_var_exp = np.cumsum(var_exp) # varianza acumulada por los primeros n PCs
var_exp

El objeto PCA sirve además para pasar de la representación en las dimensiones originales a la de las dimensiones en el espacio de los componentes principales encontrados, a partir de su método transform:

In [ ]:
dataPca = pca.transform(dataStd.iloc[:, 0:6])

Veamos gráficamente la cantidad de información correspondiente a cada componente principal:

In [ ]:
plt.figure(figsize=(15, 7))
plt.bar(range(len(var_exp)), var_exp, alpha=0.3333, align='center', label='Varianza explicada por cada PC', color = 'g')
plt.step(range(len(cum_var_exp)), cum_var_exp, where='mid',label='Varianza explicada acumulada')
plt.ylabel('Porcentaje de varianza explicada')
plt.xlabel('Componentes principales')
plt.legend(loc='best')
plt.show()

In [ ]:
np.sum(pca.explained_variance_ratio_[0:3])

Encontramos que los primeros 3 componentes conservan el 81.6% de la información original, y los primeros 4 el 93.2%.
Vamos a quedarnos solo con los 3 primeros PCs.

In [ ]:
dataPca = dataPca[:,0:3]

In [ ]:
dataPca[0:5]

Vamos a ver los puntos en el nuevo sistema de representación dado por los componentes principales.
Creamos una función que permite plotear tanto los puntos de los datos como los loadings de las variables originales (tomada de https://stackoverflow.com/questions/39216897/plot-pca-loadings-and-loading-in-biplot-in-sklearn-like-rs-autoplot).
Esto nos permitirá entender mejor la relación entre componentes principales y variables originales.

In [ ]:
def biplot(data, loadings, index1, index2, labels=None):
    plt.figure(figsize=(15, 7))
    xs = data[:,index1]
    ys = data[:,index2]
    n=loadings.shape[0]
    scalex = 1.0/(xs.max()- xs.min())
    scaley = 1.0/(ys.max()- ys.min())
    plt.scatter(xs*scalex,ys*scaley)
    for i in range(n):
        plt.arrow(0, 0, loadings[i,index1], loadings[i,index2],color='r',alpha=0.5)
        if labels is None:
            plt.text(loadings[i,index1]* 1.15, loadings[i,index2] * 1.15, "Var"+str(i+1), color='g', ha='center', va='center')
        else:
            plt.text(loadings[i,index1]* 1.15, loadings[i,index2] * 1.15, labels[i], color='g', ha='center', va='center')
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    plt.xlabel("PC{}".format(index1))
    plt.ylabel("PC{}".format(index2))
    plt.grid() 

Veamos como nos va con los primeros dos componentes principales:

In [ ]:
biplot(dataPca, pca.components_, 0, 1, ['Fresh', 'Milk', 'Grocery', 'Frozen', 'Detergents_Paper', 'Delicassen'])

In [ ]:
biplot(dataPca, pca.components_, 0, 2, ['Fresh', 'Milk', 'Grocery', 'Frozen', 'Detergents_Paper', 'Delicassen'])

In [ ]:
dataStd.columns[0:6]

In [ ]:
pca.components_

Podemos decir que:
- El componente PC1 representa positivamente las compras de leche en su sentido positivo, y negativamente las compras en Groceries y Frozen. Las otras variables no tienen mayor incidencia.
- El componente PC2 representa sobretodo las compras de Detergentes/Papel y Fresh (positivamente)
- El componente PC3 representa sobretodo las compras de Delicatessen y Fresh (positivamente), y Detergentes/Papel y Frozen (negativamente)

Ahora que ya entendemos el significado de los componentes principales, podemos proseguir a un clustering de los registros en el espacio reducido:

In [ ]:
dataPca = pd.DataFrame(dataPca)
dataPca.columns=['PC1', 'PC2', 'PC3']

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0, n_init=10)
kmeans.fit(dataPca)
clusters = kmeans.labels_

In [ ]:
dataPca['Cluster']= clusters
counter=Counter(clusters)
print(counter)

In [ ]:
fig = plt.figure(figsize=(18,15))
i=1
for var in dataPca.columns[0:3]:
    ax = fig.add_subplot(math.ceil(len(var_num)/2), 2, i)
    sns.kdeplot(dataPca.loc[dataPca.Cluster==0][var], shade=True, color='r', ax=ax);
    sns.kdeplot(dataPca.loc[dataPca.Cluster==1][var], shade=True, color='b', ax=ax);
    sns.kdeplot(dataPca.loc[dataPca.Cluster==2][var], shade=True, color='g', ax=ax);
    plt.title(var)
    plt.legend(['Cluster 0', 'Cluster 1', 'Cluster 2'])
    i+=1

Vemos que con K=3, El PC1 sirve para separar bien los puntos del cluster rojo (0), el PC2 sirve para distinguir el cluster verde (2). El cluster azul (1) no se puede separar directamente de los demas a través de uno de los PCs, pero si al considerar los 3 PCs.

Veamoslos en scatterplots para entender mejor las diferencias:

In [ ]:
fig = plt.figure(figsize=(15,10))
colorPalette = ["r", "b", "g"]
ax = fig.add_subplot(2, 2, 1)
sns.scatterplot(x="PC1", y="PC2", hue="Cluster", data=dataPca, ax=ax, palette=colorPalette, s=100, alpha=0.5)
plt.title("PC1 vs. PC2")
ax = fig.add_subplot(2, 2, 2)
sns.scatterplot(x="PC1", y="PC3", hue="Cluster", data=dataPca, ax=ax, palette=colorPalette, s=100, alpha=0.5)
plt.title("PC1 vs. PC3")
ax = fig.add_subplot(2, 2, 3)
sns.scatterplot(x="PC2", y="PC3", hue="Cluster", data=dataPca, ax=ax, palette=colorPalette, s=100, alpha=0.5)
plt.title("PC2 vs. PC3")
plt.show()

Vemos que con K=3, en el plot de los 2 primeros PCs podemos separar bien los 3 clusters.
Recordemos que el PC1 representa positivamente las compras de leche en su sentido positivo, y negativamente las compras en Groceries y Frozen, y que el componente PC2 representa sobretodo las compras de Detergentes/Papel y Fresh (positivamente).

**Nota**: Realizar la determinación del número de cluster puede hacerse tanto en el espacio de representación original (ya estandarizado) como en el de los componentes principales (considerandolos todos). Los resultados serán los mismos, ya que tanto el método del codo como el de la silueta se basan en cálculos de distancias, que se conservan después de la transformación en componentes temporales, que no es más que una rotación de los ejes.
